In [ ]:
#Librerias
import pandas as pd
import numpy as np
import logging
from pathlib import Path
import sys
from io import StringIO

In [ ]:
"""" 
Script para crear la dimensión de violaciones sanitarias a partir de datos procesados.
Este script carga un archivo CSV de inspecciones sanitarias, extrae información única sobre violaciones
"""
# --- Configuración de Logging ---
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(name)s.%(funcName)s: %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
log = logging.getLogger(__name__)

# --- Definición de Rutas ---
try:
    # Establecemos las rutas base del proyecto
    RUTA_BASE = Path(r"H:\git\proyecto grupal 2\Yelp-Gmaps-Proyecto-DS").resolve()
    RUTA_PROCESSED = RUTA_BASE / "data" / "processed"
    RUTA_DIM = RUTA_PROCESSED / "dim"

    # Ruta del archivo de salubridad procesado 
   
    RUTA_SALUBRIDAD_PROC = (RUTA_PROCESSED / "salubridad_restaurantes_con_ids.csv").resolve()

    # Ruta para la dimensión de violaciones
    RUTA_SALIDA_DIM_VIOLACION = (RUTA_DIM / "dim_violacion.csv").resolve()

    # Creamos directorios necesarios
    RUTA_PROCESSED.mkdir(parents=True, exist_ok=True)
    RUTA_DIM.mkdir(parents=True, exist_ok=True)

except Exception as e:
    log.exception(f"Error crítico al definir rutas: {e}")
    sys.exit(1)

# --- Constantes de Nombres de Columna ---
# Define los nombres EXACTOS de las columnas en RUTA_SALUBRIDAD_PROC
# *** AJUSTA SI SON DIFERENTES EN TU ARCHIVO ***
COLUMNA_CODIGO_VIOLACION = "codigo_violacion"
COLUMNA_DESC_VIOLACION = "descripcion_violacion"
COLUMNA_ES_CRITICA = "es_critica" # Columna que indica si es 'Critical' o 'Not Critical'

# --- Funciones ---
"""Cargamos el archivo CSV procesado, manejando adecuadamente errores."""
def cargar_datos_procesados(ruta_archivo: Path) -> pd.DataFrame | None:
    
    log.info(f"Cargando archivo procesado desde: {ruta_archivo}")
    if not ruta_archivo.is_file():
        log.error(f"Archivo no encontrado: {ruta_archivo}")
        return None
    try:
        # Definir tipos de datos esperados para las columnas
        dtypes_estimados = {
            COLUMNA_CODIGO_VIOLACION: str,
            COLUMNA_DESC_VIOLACION: str,
            COLUMNA_ES_CRITICA: str 
        }
        df = pd.read_csv(ruta_archivo, dtype=dtypes_estimados, low_memory=False, keep_default_na=False) # keep_default_na=False ayuda con booleanos
        log.info(f"Archivo cargado: {df.shape[0]} filas, {df.shape[1]} columnas.")
        return df
    except ValueError as ve:
        # Error común si una columna esperada en dtype no existe
         log.error(f"Error de 'ValueError' al cargar CSV. ¿Existen las columnas especificadas en dtype? Error: {ve}")
         # Intentamos cargar sin especificar dtypes
         try:
             log.warning("Reintentando carga sin especificar dtypes...")
             df = pd.read_csv(ruta_archivo, low_memory=False)
             log.info(f"Archivo cargado sin dtypes específicos: {df.shape[0]}x{df.shape[1]}.")
             return df
         except Exception as e_fallback:
             log.exception(f"Fallo carga fallback: {e_fallback}")
             return None
    except Exception as e:
        log.exception(f"Error al cargar CSV desde '{ruta_archivo}': {e}")
        return None

    """
    Extraemos la información única de violaciones sanitarias, crea una dimensión y la guarda como CSV.

    """
def crear_y_guardar_dimension_violacion(df_inspecciones: pd.DataFrame, ruta_salida: Path) -> bool:

    log.info("Creando dimensión de violaciones sanitarias...")

    # Definimos las columnas fuente que necesitamos
    cols_fuente = [COLUMNA_CODIGO_VIOLACION, COLUMNA_DESC_VIOLACION, COLUMNA_ES_CRITICA]

    # Verificamos que existan en el DataFrame
    if not all(col in df_inspecciones.columns for col in cols_fuente):
        log.error(f"Faltan columnas requeridas ({cols_fuente}) en el DataFrame. No se puede crear dimensión.")
        return False

    try:
        # 1. Seleccionamos columnas y eliminamos filas sin código de violación
        df_viol = df_inspecciones[cols_fuente].copy()
        df_viol.dropna(subset=[COLUMNA_CODIGO_VIOLACION], inplace=True)
        # También quitar si el código es un string vacío o solo espacios
        df_viol = df_viol[df_viol[COLUMNA_CODIGO_VIOLACION].astype(str).str.strip() != '']
        log.info(f"Filas con violaciones encontradas: {len(df_viol)}")
        if df_viol.empty:
             log.warning("No se encontraron filas con códigos de violación válidos.")
             return False # No tiene sentido continuar

        # 2. Limpiamos textos y estandarizamos la columna 'es_critica' a Booleano
        log.info("Limpiando textos y estandarizando criticidad...")
        df_viol[COLUMNA_CODIGO_VIOLACION] = df_viol[COLUMNA_CODIGO_VIOLACION].astype(str).str.strip()
        df_viol[COLUMNA_DESC_VIOLACION] = df_viol[COLUMNA_DESC_VIOLACION].astype(str).str.strip()

        
        df_viol['es_critica_bool'] = df_viol[COLUMNA_ES_CRITICA].astype(str).str.contains('Critical', case=False, na=False)
        log.info(f"Valores únicos en '{COLUMNA_ES_CRITICA}' original: {df_inspecciones[COLUMNA_ES_CRITICA].unique()}")
        log.info(f"Valores únicos en 'es_critica_bool' mapeada: {df_viol['es_critica_bool'].unique()}")


        # 3. Encontramos combinaciones únicas y verificar consistencia del código
        log.info("Buscando combinaciones únicas de violaciones...")
        cols_unicas = [COLUMNA_CODIGO_VIOLACION, COLUMNA_DESC_VIOLACION, 'es_critica_bool']
        violaciones_unicas = df_viol[cols_unicas].drop_duplicates()
        log.info(f"Encontradas {len(violaciones_unicas)} combinaciones únicas.")

        # Verificamos si algún código tiene múltiples descripciones/criticidades
        duplicados_en_codigo = violaciones_unicas[violaciones_unicas.duplicated(subset=[COLUMNA_CODIGO_VIOLACION], keep=False)]
        if not duplicados_en_codigo.empty:
            log.warning(f"¡INCONSISTENCIA EN DATOS FUENTE! Se encontraron códigos de violación con múltiples descripciones/criticidades:")
            log.warning(f"\n{duplicados_en_codigo.sort_values(by=COLUMNA_CODIGO_VIOLACION).to_string()}")
            dim_violacion = violaciones_unicas.drop_duplicates(subset=[COLUMNA_CODIGO_VIOLACION], keep='first')
            log.warning(f"Se mantendrá la primera descripción/criticidad encontrada para cada código duplicado. Registros finales: {len(dim_violacion)}")
        else:
            log.info("Verificación de consistencia OK: Cada código de violación tiene una única descripción/criticidad.")
            dim_violacion = violaciones_unicas # Ya son únicos por código

        # 4. Renombramos las columnas para la dimensión final
        dim_violacion.rename(columns={'es_critica_bool': 'es_critica'}, inplace=True)

        # 5. Ordenamos por código de violación
        dim_violacion.sort_values(by=COLUMNA_CODIGO_VIOLACION, inplace=True)
        dim_violacion.reset_index(drop=True, inplace=True)

        # 6. Nos aceguramos que los tipos de datos sean correctos
        dim_violacion[COLUMNA_CODIGO_VIOLACION] = dim_violacion[COLUMNA_CODIGO_VIOLACION].astype('string')
        dim_violacion[COLUMNA_DESC_VIOLACION] = dim_violacion[COLUMNA_DESC_VIOLACION].astype('string')
        dim_violacion['es_critica'] = dim_violacion['es_critica'].astype('boolean') # Tipo nullable boolean

        log.info(f"Dimensión de violaciones final creada con {len(dim_violacion)} registros.")
        log.debug(f"Tipos de datos finales:\n{dim_violacion.dtypes}")

        # 7. Guardamos la dimensión
        log.info(f"Guardando dimensión de violaciones en: {ruta_salida}")
        ruta_salida.parent.mkdir(parents=True, exist_ok=True)
        dim_violacion.to_csv(ruta_salida, index=False, encoding='utf-8-sig')
        log.info("Dimensión de violaciones guardada exitosamente.")
        return True

    except Exception as e:
        log.exception(f"Error creando o guardando la dimensión de violaciones: {e}")
        return False

# --- Flujo Principal ---
def main_crear_dim_violacion():
    """Orquestamos la creación y guardado de la dimensión de violaciones."""
    log.info("================================================")
    log.info("=== INICIO: Crear Dimensión Violaciones Sanitarias ===")
    log.info("================================================")

    # 1. Validar archivo de entrada
    log.info(f"Verificando archivo de entrada: {RUTA_SALUBRIDAD_PROC}")
    if not RUTA_SALUBRIDAD_PROC.is_file():
        log.critical(f"Abortando: Archivo de entrada no encontrado: {RUTA_SALUBRIDAD_PROC}")
        sys.exit(1)
    log.info("Archivo de entrada encontrado.")

    # 2. Cargar datos de inspecciones
    df_inspecciones = cargar_datos_procesados(RUTA_SALUBRIDAD_PROC)
    if df_inspecciones is None or df_inspecciones.empty:
        log.critical("Fallo al cargar datos de inspecciones. Abortando.")
        sys.exit(1)

    # 3. Crear y guardar la dimensión
    exito = crear_y_guardar_dimension_violacion(df_inspecciones, RUTA_SALIDA_DIM_VIOLACION)

    log.info("================================================")
    if exito:
        log.info("=== Proceso completado exitosamente ===")
        log.info(f"Archivo de dimensión creado en: {RUTA_SALIDA_DIM_VIOLACION}")
    else:
        log.error("=== Proceso falló al crear/guardar la dimensión de violaciones ===")
    log.info("================================================")


if __name__ == "__main__":
    main_crear_dim_violacion()

2025-04-19 14:44:39 [INFO] __main__.main_crear_dim_violacion: ================================================
2025-04-19 14:44:39 [INFO] __main__.main_crear_dim_violacion: === INICIO: Crear Dimensión Violaciones Sanitarias ===
2025-04-19 14:44:39 [INFO] __main__.main_crear_dim_violacion: ================================================
2025-04-19 14:44:39 [INFO] __main__.main_crear_dim_violacion: Verificando archivo de entrada: H:\git\proyecto grupal 2\Yelp-Gmaps-Proyecto-DS\data\processed\salubridad_restaurantes_con_ids.csv
2025-04-19 14:44:39 [INFO] __main__.main_crear_dim_violacion: Archivo de entrada encontrado.
2025-04-19 14:44:39 [INFO] __main__.cargar_datos_procesados: Cargando archivo procesado desde: H:\git\proyecto grupal 2\Yelp-Gmaps-Proyecto-DS\data\processed\salubridad_restaurantes_con_ids.csv
2025-04-19 14:44:41 [INFO] __main__.cargar_datos_procesados: Archivo cargado: 277686 filas, 28 columnas.
2025-04-19 14:44:41 [INFO] __main__.crear_y_guardar_dimension_violacion: Cre